# 2020语言与智能技术竞赛：事件抽取任务--方案分享(Test1:Rank17 Test2:Rank18)

# 本模型在官方PaddleHub版本Baseline上进行修改得到

官方原版地址:https://github.com/PaddlePaddle/Research/tree/master/KG/DuEE_baseline/DuEE-PaddleHub

本方案github地址:https://github.com/onewaymyway/DuEE_2020

# 本方案在官方baseline的基础上的改动

1.在网络结构上在CRF层前面增加了双向GRU层（代码见sequence_label.py中SequenceLabelTaskSP类）

2.将trigger预测结果拼接到text前面进行第二阶段的role预测(代码见data_process.py的data_process函数中model=role1的情况)，这个改动可以解决同一个句子不同event之间role重叠的问题

3.在训练上，本方案先只用train进行训练，然后再将dev放入train进行最后的训练

4.增加了简单的最终结果剔除机制(代码见datachecker.py)


## 注意

本项目代码需要使用GPU环境来运行:

<img src="https://ai-studio-static-online.cdn.bcebos.com/767f625548714f03b105b6ccb3aa78df9080e38d329e445380f505ddec6c7042" width="40%" height="40%">
<br>
<br>
并且检查相关参数设置, 例如use_gpu, fluid.CUDAPlace(0)等处是否设置正确. 

# 事件抽取任务

事件抽取任务的目标是通过给定目标事件类型和角色类型集合及句子，识别句子中所有目标事件类型的事件，并根据论元角色集合抽取事件所对应的论元。其中目标事件类型（event_type）和论元角色（role）限定了抽取的范围，例如：（event_type：胜负，role：时间，胜者，败者，赛事名称）、（event_type：夺冠，role：夺冠事件，夺冠赛事，冠军）。最终将抽取的所有事件论元按如下形式进行输出: {“id”: “id1”, “event_list”: [{“event_type”:“T1”, “arguments”: [{“role”:“R1”, “argument”:“A1”},…]}, {“event_type”:“T2”, “arguments”: [{“role”:“R2”, “argument”:“A2”},…]}]}，比赛将对参赛者最终输出的论元列表进行评估。

# 方案
本方案分两个阶段，第一个阶段为事件类型抽取，第二阶段为事件论元抽取，两个阶段的模型都是 预训练模型+GRU+CRF进行序列标注，接下来用一个例子来大概讲讲流程。

假设当前要预测的句子为：

历经4小时51分钟的体力、意志力鏖战，北京时间9月9日上午纳达尔在亚瑟·阿什球场，以7比5、6比3、5比7、4比6和6比4击败赛会5号种子俄罗斯球员梅德韦杰夫，夺得了2019年美国网球公开赛男单冠军。

## 事件类型抽取
当输入例子句子之后，事件类型阶段的输出结果类似于是

历经4小时51分钟的体力、意志力鏖战，北京时间9月9日上午纳达尔在亚瑟·阿什球场，以7比5、6比3、5比7、4比6和6比4击(B-竞赛行为-胜负)败(I-竞赛行为-胜负)赛会5号种子俄罗斯球员梅德韦杰夫，夺得了2019年美国网球公开赛男单冠(B-竞赛行为-夺冠)军(I-竞赛行为-夺冠)。

(击,B-竞赛行为-胜负)(败,I-竞赛行为-胜负)

(冠,B-竞赛行为-夺冠)(军,I-竞赛行为-夺冠)

其它字符的标注为O

然后就得到了这个句子的两个事件类型[竞赛行为-胜负,竞赛行为-夺冠]

结果文件存在data1/test1.json.trigger.pred文件中

## 论元抽取

与官方baseline不同，本模型方案将事件拼接到句子前面进行论元抽取，比如刚才的例子，第二阶段将分别进行两次不同的预测

1.竞赛行为-胜负：历经4小时51分钟的体力、意志力鏖战，北京时间9月9日上午纳达尔在亚瑟·阿什球场，以7比5、6比3、5比7、4比6和6比4击败赛会5号种子俄罗斯球员梅德韦杰夫，夺得了2019年美国网球公开赛男单冠军。

输出结果：

竞赛行为-胜负：历经4小时51分钟的体力、意志力鏖战，北(B-时间)京(I-时间)时(I-时间)间(I-时间)9(I-时间)月(I-时间)9(I-时间)日(I-时间)上(I-时间)午(I-时间)纳(B-胜者)达(I-胜者)尔(I-胜者)在亚瑟·阿什球场，以7比5、6比3、5比7、4比6和6比4击败赛会5(B-败者)号(I-败者)种(I-败者)子(I-败者)俄(I-败者)罗(I-败者)斯(I-败者)球(I-败者)员(I-败者)梅(I-败者)德(I-败者)韦(I-败者)杰(I-败者)夫(I-败者)，夺得了2(B-赛事名称)0(I-赛事名称)1(I-赛事名称)9(I-赛事名称)年(I-赛事名称)美(I-赛事名称)国(I-赛事名称)网(I-赛事名称)球(I-赛事名称)公(I-赛事名称)开(I-赛事名称)赛(I-赛事名称)男单冠军。

2.竞赛行为-夺冠：历经4小时51分钟的体力、意志力鏖战，北京时间9月9日上午纳达尔在亚瑟·阿什球场，以7比5、6比3、5比7、4比6和6比4击败赛会5号种子俄罗斯球员梅德韦杰夫，夺得了2019年美国网球公开赛男单冠军。

输出结果：

竞赛行为-夺冠：历经4小时51分钟的体力、意志力鏖战，北(B-时间)京(I-时间)时(I-时间)间(I-时间)9(I-时间)月(I-时间)9(I-时间)日(I-时间)上(I-时间)午(I-时间)纳(B-冠军)达(I-冠军)尔(I-冠军)在亚瑟·阿什球场，以7比5、6比3、5比7、4比6和6比4击败赛会5号种子俄罗斯球员梅德韦杰夫，夺得了(B-夺冠赛事)2(I-夺冠赛事)0(I-夺冠赛事)1(I-夺冠赛事)9(I-夺冠赛事)年(I-夺冠赛事)美(I-夺冠赛事)国(I-夺冠赛事)网(I-夺冠赛事)球(I-夺冠赛事)公(I-夺冠赛事)开(I-夺冠赛事)赛(I-夺冠赛事)男单冠军。

训练的时候也是将同一个句子的不同事件拆开转换成不同的句子进行训练，训练的时候一个事件里只有这个事件的论元，忽略这个句子里其它事件的论元。

## 生成最终结果

最终结果根据论元抽取结果生成，将相同句子的不同事件合到同一个结果里。

```
{
    "id":"6a10824fe9c7b2aa776aa7e3de35d45d",
    "event_list":[
        {
            "event_type":"竞赛行为-胜负",
            "arguments":[
                {
                    "role":"时间",
                    "argument":"北京时间9月9日上午"
                },
                {
                    "role":"胜者",
                    "argument":"纳达尔"
                },
                {
                    "role":"败者",
                    "argument":"5号种子俄罗斯球员梅德韦杰夫"
                },
                {
                    "role":"赛事名称",
                    "argument":"2019年美国网球公开赛"
                }
            ]
        },
        {
            "event_type":"竞赛行为-夺冠",
            "arguments":[
                {
                    "role":"时间",
                    "argument":"北京时间9月9日上午"
                },
                {
                    "role":"夺冠赛事",
                    "argument":"2019年美国网球公开赛"
                },
                {
                    "role":"冠军",
                    "argument":"纳达尔"
                }
            ]
        }
    ]
}
```

# 题外话
本方案是最终参赛的方案，在此之外还有一个魔改了PaddleHub的方案，但是效果并没有这个好，但是那个方案可以作为一个魔改PaddleHub的好案例，有时间把那个方案也分享一下。基本思路和这个有点相似，也是将事件信息加到第二阶段的论元抽取，不同的是，这个方案是把文字直接加到了句子前面，另一个方案是把事件做了onehot编码加到了每个词向量上，还将分词信息也加到了词向量上。和这个方案相比，另一个方案实现了一套往PaddleHub模型中的词向量追加Feature的机制（其实我没想明白为啥那个方案效果没这个方案好：（）。

本方案排名并不靠前，分享的目的还是希望可以抛砖引玉，希望也能看到前排的大佬们分享一下方案（虽然赛后群里有大佬大致分享了，但是还是想看更详细的分享：））。

感觉本次参加比赛的收获挺大的，大概有以下几点：

1.大致熟悉了PaddleHub这个框架

2.认识了好多大佬，学习到了好多新的知识点

3.发现了竟然还有AIStudio这么一个可以白嫖GPU的好地方（因为没GPU其实这是我第一次玩深度学习打比赛）

# 如何在AIStudio把这个方案跑起来

fork这个项目，然后一直往后执行就可以了 : )

PS.

为了训练方便，本项目接了VisualDL,所以在训练的时候打开VisualDL的页面是可以看到统计信息的

比如:
![](https://ai-studio-static-online.cdn.bcebos.com/a03481088c6040bb9adb0036d08e41c9cbeb04648dde49f4aa7a74b15692bbd8)

## 如何打开VisualDL

# Notebooks项目访问URL

比如你的notebook网页地址为
url_notebook = 'http://aistudio.baidu.com/user/30799/33852/notebooks/33852.ipynb?redirects=1'

# 替换后visualdl访问URL
url_visualdl = 'http://aistudio.baidu.com/user/30799/33852/visualdl'

访问后面那个网址就能打开VisualDL了

In [ ]:
#从github拉代码
#如果是fork的这个项目，这一步就不用执行了，如果是自己新开的项目可以执行这一步
#!svn checkout https://github.com/PaddlePaddle/Research.git/trunk/KG/DuEE_baseline/DuEE-PaddleHub/ ./baseline
!svn checkout https://github.com/onewaymyway/DuEE_2020.git/trunk ./baseline

In [ ]:
# 切换到代码目录
%cd baseline/

/home/aistudio/baseline


In [ ]:
#安装依赖
!pip install -r ./requirements.txt

In [18]:
#这个代码不用执行！！！
#打包文件夹下的文件 用于本地备份 排除数据目录和模型目录

!zip -r mb.zip * -x "./models/*" -x "./model/*" -x "./data/*" -x "./data1/*" -x "./orzdata/*"

In [ ]:
#拉取最新的比赛数据

!rm -rf ./orzdata
!mkdir ./orzdata
!rm -rf ./data
!mkdir ./data
%cd ./orzdata

#训练数据 dev数据 test1
!wget -O train.zip https://dataset-bj.cdn.bcebos.com/event_extraction/train_data.json.zip
!unzip train.zip
!wget -O dev.zip https://dataset-bj.cdn.bcebos.com/event_extraction/dev_data.json.zip
!unzip dev.zip
!wget -O test.zip https://dataset-bj.cdn.bcebos.com/event_extraction/test1_data.json.zip
!unzip test.zip

!cp -r ./dev_data/dev.json ../data/dev.json
!cp -r ./train_data/train.json ../data/train.json
!cp -r ./test1_data/test1.json ../data/test1.json

#schema数据
!wget -O schema.zip https://ai.baidu.com/file/9C92719AF96D4DDB96477BFBE1435262
!unzip schema.zip

!cp -r ./event_schema/event_schema.json ../data/event_schema.json

#test2
!wget -O test2.zip https://dataset-bj.cdn.bcebos.com/lic2020/test2_data.json.zip
!unzip test2.zip
!cp -r ./test2.json ../data/test2.json

%cd ../

In [ ]:
#将数据拷到data1防止做实验把数据覆盖了
!cp -r ./data ./data1

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

#为了做实验方便封装的脚本调用

import os
import subprocess


def executeSh(sh):
    '''
    执行sh命令的封装
    '''
    print("sh:",sh)
    v=os.popen(sh)
    lines=v.readlines()
    for line in lines:
        print(line)
        #print(line.decode("utf-8"))
        #print(line.decode("gb2312"))
    v.close()
def executeShs(shlist):
    '''
    执行多个sh命令的封装
    '''
    for sh in shlist:
        executeSh(sh)
    
def savebestToTemp(modelpath):
    '''
    将modelpath/best_model复制到modelpath/curtest
    主要用于保存和测试当前的best_model,防止best_model被覆盖无法重现测试的结果
    '''
    tar=modelpath+"/curtest"
    src=modelpath+"/best_model/."
    sh="rm -rf "+tar
    executeSh(sh)
    sh="mkdir "+tar
    executeSh(sh)
    sh="cp -r "+src+" "+tar
    executeSh(sh)

def checkPredFile(filePath):
    '''
    删除结果中不合理的元素
    '''
    executeSh("python datachecker.py --targetFile "+filePath)

def run_gru_model(triggermodel,testFile,saveFile):
    '''
    一次性跑完从trigger预测到role预测再到输出最终结果
    triggermodel:使用的trigger模型目录
    testFile:要预测的文件
    saveFile:最终输出文件
    '''
    shs=[
        "sh run_trigger_gru_predict_withmodel.sh 0 ./data1/ models/trigger_gru  {0} {1}".format(testFile,triggermodel),
        "python data_process.py --action predict_data_process2 --trigger_file data1/{0}.trigger.pred  --schema_file data/event_schema.json --save_path data1/{0}_triggerpred.json".format(testFile,triggermodel),
        "sh run_role1_gru_predict.sh 0 ./data1/ models/rolegru {0}_triggerpred.json".format(testFile,triggermodel),
        "python data_process.py --action predict_data_process3 --role_file data1/{0}_triggerpred.json.role1.pred --schema_file data1/event_schema.json --save_path data1/{2}".format(testFile,triggermodel,saveFile)
    ]
    #print(shs)
    executeShs(shs)

def run_gru_modelWithOutmark(triggermodel,testFile,outmark=""):
    '''
    生成带mark的预测文件
    主要用于模型之间的融合
    仅用于实验，比赛最终结果没用模型融合
    '''
    shs=[
        "sh run_trigger_gru_predict_withmodel.sh 0 ./data1/ models/trigger_gru  {0} {1}".format(testFile,triggermodel),
        "python data_process.py --action predict_data_process2 --trigger_file data1/{0}.trigger.pred  --schema_file data/event_schema.json --save_path data1/{0}{2}_triggerpred.json".format(testFile,triggermodel,outmark)
    ]
    #print(shs)
    executeShs(shs)  

def runRolePredict(triggerfile="test1_triggerpred.json",rolemodel="models/rolegru",gru=False,predictmodel="best_model",outmark="",finalFile=None,ifCheck=False,data_dir="./data1"):
    '''
    根据trigger预测结果进行role预测并生成最终的结果
    '''
    sh="python sequence_label.py --num_epoch 30 \
    --learning_rate 3e-5 \
    --data_dir {data_dir} \
    --schema_path {data_dir}/event_schema.json \
    --train_data {data_dir}/train.json \
    --dev_data {data_dir}/dev.json \
    --test_data {data_dir}/dev.json \
    --predict_data {data_dir}/{predictfile} \
    --do_train False \
    --do_predict False \
    --do_predict2 True \
    --do_model role1 \
    --add_gru {gru} \
    --predictmodel {predictmodel} \
    --max_seq_len 256 \
    --batch_size 8 \
    --model_save_step 3000 \
    --eval_step 200 \
    --checkpoint_dir {ckpt_dir}".format(data_dir=data_dir,predictfile=triggerfile,gru=gru,ckpt_dir=rolemodel,predictmodel=predictmodel)
    if outmark:
        sh+=" --outmark "+outmark
    executeSh(sh)
    if finalFile:
        makefinalFile(("{data_dir}/"+triggerfile+".role1{outmark}.pred").format(data_dir=data_dir,outmark=outmark),finalFile)
        if ifCheck:
            checkPredFile(finalFile)

def makefinalFile(rolePredFile="data1/test1_triggerpred_md.json.role1.pred",savefile="data1/test1fn_pred.json"):
    '''
    根据role预测结果生成最终提交数据
    '''
    sh="python data_process.py --action predict_data_process3 --role_file {0} --schema_file data1/event_schema.json --save_path {1}".format(rolePredFile,savefile)
    #sh="sh makefinalpred.sh {0} {1}".format(rolePredFile,savefile)
    executeSh(sh)

In [ ]:
#为了快速跑流程 代码中的预训练模型改成了ernie_tiny
#实际比赛用的是chinese-roberta-wwm-ext-large
#要达到比赛的分数效果可以在sequence_label.py中将预训练模型改为chinese-roberta-wwm-ext-large
#为了简洁，删掉了很多做实验的代码和脚本
#训练建议到终端里进行训练，因为在终端训练关了浏览器再进还能看到输出 ： ）

In [ ]:
#训练trigger_gru 加了gru层版本trigger识别模型
!sh run_trigger_gru.sh 0 ./data1/ models/trigger_gru

In [ ]:
#训练trigger_gru_mix 将dev数据也放入训练
!sh run_trigger_gru_mix.sh 0 ./data1/ models/trigger_gru

In [ ]:
#trigger_gru 预测
#预测结果存在data1/test1.json.trigger.pred
!sh run_trigger_gru_predict.sh 0 ./data1/ models/trigger_gru  test1.json

In [ ]:
#根据trigger_gru预测结果数据生成role预测需要的数据
#结果文件为data1/test1_triggerpred.json
#具体代码见data_process.py predict_data_process2函数
!python data_process.py --action predict_data_process2 --trigger_file data1/test1.json.trigger.pred  --schema_file data/event_schema.json --save_path data1/test1_triggerpred.json

In [ ]:
#训练role1 gru 训练加了gru层的role识别模型
!sh run_role1_gru.sh 0 ./data1/ models/rolegru

In [15]:
#训练role1_gru mix 将dev也加入训练
!sh run_role1_gru_mix.sh 0 ./data1/ models/rolegru

In [16]:
#role1_gru预测
#结果文件为data1/test1_triggerpred.json.role1.pred
!sh run_role1_gru_predict.sh 0 ./data1/ models/rolegru test1_triggerpred.json
#role1 生成最终提交结果
#结果文件为data1/test1fn_pred.json
!python data_process.py --action predict_data_process3 --role_file data1/test1_triggerpred.json.role1.pred --schema_file data1/event_schema.json --save_path data1/test1fn_pred.json

In [17]:
#从结果中删除明显不合理的结果，这一步有一点提升，不做也没大影响
#结果文件为data1/test1fn_pred_md.json
checkPredFile("./data1/test1fn_pred.json")

In [ ]:
#AIStudio更新代码编辑器之后出现编辑sh文件后无法执行sh文件的问题
#需要将每行的\r手动去掉才能正常执行

from nlputils import readFile,saveFile
def adptSHFile(filePath):
    txt=readFile(filePath)
    txt=txt.replace("\r","")
    saveFile(filePath,txt)

adptSHFile("run_role1_gru.sh")
adptSHFile("run_role1_gru_mix.sh")
adptSHFile("run_role1_gru_eval.sh")
adptSHFile("run_role1_gru_predict.sh")
adptSHFile("run_trigger_gru.sh")
adptSHFile("run_trigger_gru_predict.sh")
adptSHFile("run_trigger_gru_mix.sh")
adptSHFile("run_trigger_gru_predict_withmodel.sh")
print("ok")

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 